In [16]:
import re

from dotenv import load_dotenv
from anthropic import AsyncAnthropic

import os

load_dotenv(os.path.abspath(os.path.join(os.getcwd(), "../.env")))

client = AsyncAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))


async def get_messages(prompt, system_prompt="") -> str:
    response = await client.messages.create(
        model="claude-3-5-haiku-20241022",
        messages=prompt,
        max_tokens=1024 * 1,
        temperature=0.7,
        system=system_prompt
    )

    return response


In [17]:
import asyncio
import nest_asyncio
from collections import Counter

nest_asyncio.apply()


async def get_self_consistency_completion(prompt, system_prompt = "", num_responses=5) -> str:
    coros = [get_messages(prompt, system_prompt) for _ in range(num_responses)]
    response = await asyncio.gather(*coros)
    return analyze_response(response)

def analyze_response(response):
    texts = [r.content[0].text for r in response]
    answers = [extract_answer(t) for t in texts]
    frequency = Counter(answers)
    most_common = frequency.most_common(1)[0][0]

    return {
        "most_common": most_common,
        "frequency": dict(frequency),
        "total_responses": len(response),
    }


def extract_answer(text):
    match = re.search(r"<answer>\s*(.*?)\s*</answer>", text, re.DOTALL)

    if match:
        return match.group(1).strip()
    return text.strip()

In [20]:
system_prompt = """
당신은 복잡한 논리퍼즐과 문제 해결에 능숙한 AI 모델입니다.
주어진 퍼즐을 면밀히 분석하고, 단계별로 논리적인 추론을 통해 해결 방법을 도출하세요,
여러가지 접근방법을 고려하여, 가장 일관되고 합리적인 답변을 제공하세요.
추론 과정은 <thinking>, 정답은 <answer> 태그 안에 작성해주세요.
"""

messages = [
    {
        "role": "user",
        "content": """
        한 남자가 강을 건너야 하는데, 그에게는 늑대, 염소, 양배추가 있습니다.
        보트에는 남자와 함께 하나의 항목만 실을 수 있습니다.
        늑대와 염소가 함께 두면, 늑대가 염소를 먹습니다.
        염소와 양배추가 함께 두면, 염소가 양배추를 먹습니다.
        남자는 모든 항목을 강 건너편으로 옮겨야한다고 했을 때, 세번째 보트에는 남자와 무엇이 타고 있을까요?
        단 하나의 정답만 말해주세요.
        """
    }
]

print(asyncio.run(get_self_consistency_completion(messages, system_prompt)))

{'most_common': '염소', 'frequency': {'염소': 8, '양배추': 1, '늑대': 1}, 'total_responses': 10}


In [25]:
async def get_universal_consistency_completion(prompt, system_prompt = "", num_responses=5) -> str:
    coros = [get_messages(prompt, system_prompt) for _ in range(num_responses)]
    response = await asyncio.gather(*coros)
    for (i,r) in enumerate(response):
        print(f"{i}: {r.content[0].text}")

    return await select_consistent_response(response)

async def select_consistent_response(response):
    response_texts = [r.content[0].text for r in response]
    formatted = "\n\n".join([f"Response {i+1}:\n<thinking>{extract_thinking(r)}\n</thinking>\n<answer>{extract_answer(r)}</answer>" for (i,r) in enumerate(response_texts)])

    messages = [
        {
            "role": "user",
            "content": f"""
            <reference> 태그 내의 여러 답변들을 평가해보고, 이들 중에서 가장 일관성 있는 답변을 선택해주세요.
            선택한 답변을 응답으로 주세요.

            <reference>
            {formatted}
            </reference>
            """,
        }
    ]

    response = await get_messages(messages)
    return response.content[0].text


def extract_thinking(text):
    match = re.search("<thinking>(.*?)</thinking>", text, re.DOTALL)

    if match:
        return match.group(1).strip()
    return text.strip()

In [26]:
messages = [
    {
        "role": "user",
        "content": """
        한 남자가 강을 건너야 하는데, 그에게는 늑대, 염소, 양배추가 있습니다.
        보트에는 남자와 함께 하나의 항목만 실을 수 있습니다.
        늑대와 염소가 함께 두면, 늑대가 염소를 먹습니다.
        염소와 양배추가 함께 두면, 염소가 양배추를 먹습니다.
        남자는 모든 항목을 강 건너편으로 옮겨야한다고 했을 때, 어떻게 하면 모두가 무사히 강 건너편에 도착할 수 있을까요?
        """
    }
]

asyncio.run(get_universal_consistency_completion(messages))


0: 이 문제의 해결 방법은 다음과 같습니다:

1. 처음에 남자는 염소를 보트에 태워 강 건너편으로 옮깁니다.
   - 늑대와 양배추는 혼자 남아있습니다.

2. 남자는 다시 돌아와서 늑대나 양배추를 데려갈 수 없습니다.
   - 염소를 다시 데려와야 합니다.

3. 염소를 원래 쪽으로 돌려보내고, 대신 양배추를 보트에 태워 건넵니다.
   - 이제 늑대와 양배추가 강 이쪽에 남아있습니다.

4. 다시 돌아와서 염소를 데려갑니다.
   - 이제 염소가 강 건너편에 있고, 늑대와 양배추는 이쪽에 있습니다.

5. 마지막으로 늑대를 데려갑니다.

이렇게 하면 모든 항목을 안전하게 강 건너편으로 옮길 수 있습니다.

단계별 요약:
1. 염소 건너기
2. 돌아오기
3. 양배추 건너기
4. 염소 되돌리기
5. 늑대 건너기

이 방법으로 늑대가 염소를 먹거나 염소가 양배추를 먹는 상황을 피할 수 있습니다.
1: 이 문제의 해결 방법은 다음과 같습니다:

1. 처음에 남자는 염소를 먼저 강 건너편으로 옮깁니다.
   - 늑대와 양배추는 혼자 있을 수 있습니다.

2. 돌아와서 늑대나 양배추를 데려가려 하지만, 대신 염소를 다시 데리고 옵니다.
   - 이렇게 하면 처음 건너편에 남은 늑대와 양배추는 서로 해치지 않습니다.

3. 염소를 원래 쪽에 두고, 양배추를 건너편으로 옮깁니다.

4. 다시 돌아와 염소를 데리고 갑니다.

5. 이제 염소를 건너편에 두고, 늑대를 원래 쪽으로 가져옵니다.

6. 마지막으로 염소를 다시 건너편으로 옮깁니다.

이렇게 하면 모든 항목을 안전하게 강 건너편으로 옮길 수 있습니다.

단계별 요약:
1. 염소 건너기
2. 돌아와서 염소 데리고 오기
3. 양배추 건너기
4. 염소 원래 쪽에 두고 돌아오기
5. 늑대 건너기
6. 돌아와서 염소 데리고 가기
2: 이 문제의 해결 방법은 다음과 같습니다:

1단계: 염소를 먼저 건넙니다.
- 늑대와 양배추는 혼자 남습니다.

2단계: 돌아와서 늑대나 양배추를 데리고 갑니다.
- 이때 염소를 데려가면 

'여러 답변들을 비교해 본 결과, Response 1과 Response 2가 가장 일관성 있고 상세한 해결 방법을 제시하고 있습니다. 이 중에서 Response 1을 선택하겠습니다.\n\n선택한 답변은 다음과 같습니다:\n\n이 문제의 해결 방법은 다음과 같습니다:\n\n1. 처음에 남자는 염소를 보트에 태워 강 건너편으로 옮깁니다.\n   - 늑대와 양배추는 혼자 남아있습니다.\n\n2. 남자는 다시 돌아와서 늑대나 양배추를 데려갈 수 없습니다.\n   - 염소를 다시 데려와야 합니다.\n\n3. 염소를 원래 쪽으로 돌려보내고, 대신 양배추를 보트에 태워 건넵니다.\n   - 이제 늑대와 양배추가 강 이쪽에 남아있습니다.\n\n4. 다시 돌아와서 염소를 데려갑니다.\n   - 이제 염소가 강 건너편에 있고, 늑대와 양배추는 이쪽에 있습니다.\n\n5. 마지막으로 늑대를 데려갑니다.\n\n이렇게 하면 모든 항목을 안전하게 강 건너편으로 옮길 수 있습니다.\n\n단계별 요약:\n1. 염소 건너기\n2. 돌아오기\n3. 양배추 건너기\n4. 염소 되돌리기\n5. 늑대 건너기\n\n이 방법으로 늑대가 염소를 먹거나 염소가 양배추를 먹는 상황을 피할 수 있습니다.'